In [1]:
import pulp

In [4]:
import numpy as np 

In [15]:
N = 10
agents = np.zeros((N, 2))
agents[:, 0] = np.random.randint(1, 11, N)
agents[:, 1] = np.random.randint(1, 11, N)

In [16]:
agents

array([[ 3.,  4.],
       [ 5.,  4.],
       [ 7.,  3.],
       [ 8.,  2.],
       [ 7., 10.],
       [ 6.,  5.],
       [ 1.,  8.],
       [ 1.,  1.],
       [ 2.,  5.],
       [ 6.,  7.]])

In [34]:
M = 7

In [35]:
prob = pulp.LpProblem("Welfare_Maximization", pulp.LpMaximize)

In [36]:
# Create variables
x = pulp.LpVariable.dicts("Allocation", range(N), 0, 1)  # allocation per agent
p = pulp.LpVariable.dicts("Price", range(N), 0)  # price per agent
t = pulp.LpVariable.dicts("Time", range(N), 0)  # time per agent

In [37]:
# Objective function
alpha = 3
prob += pulp.lpSum([agents[i][0] * x[i] - agents[i][1] * p[i] - t[i] + alpha * p[i] for i in range(N)])

In [38]:
# Constraint: sum of the allocations <= M
prob += pulp.lpSum(x) <= M

In [39]:
for i in range(N):
    prob += agents[i][0] * x[i] - agents[i][1] * p[i] - t[i] >= 0  # utility > 0


In [40]:
# Constraints: individual rationality and incentive compatibility
for i in range(N):
    prob += agents[i][0] * x[i] - agents[i][1] * p[i] - t[i] >= 0  # utility > 0
    for j in range(N):
        if i != j:
            prob += agents[i][0] * x[i] - agents[i][1] * p[i] - t[i] >= agents[i][0] * x[j] - agents[i][1] * p[j] - t[j]  # utility from own allocation >= utility from other's allocation


In [41]:
# Solve problem
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/benjaminwittenbrink/opt/anaconda3/envs/cs360/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/64/knc2zn357c35n6k0cyxpqm_40000gn/T/f658043fbf804c78aab2c181ba1e0b6f-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/64/knc2zn357c35n6k0cyxpqm_40000gn/T/f658043fbf804c78aab2c181ba1e0b6f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 116 COLUMNS
At line 757 RHS
At line 869 BOUNDS
At line 880 ENDATA
Problem MODEL has 111 rows, 30 columns and 610 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 101 (-10) rows, 30 (0) columns and 580 (-30) elements
0  Obj -0 Dual inf 48.999988 (12)
31  Obj 36.164345 Primal inf 27.222434 (36)
38  Obj 35.899441
Optimal - objective value 35.899441
After Postsolve, objective 35.899441, infeasibilities - dual 0 (0), primal 0 (0)
Optim

1

In [42]:
# Print optimal values
for i in range(N):
    print(f"Agent {i}: Allocation = {x[i].varValue}, Price = {p[i].varValue}, Time = {t[i].varValue}")

Agent 0: Allocation = 0.67039106, Price = 0.097765363, Time = 0.083798883
Agent 1: Allocation = 0.83798883, Price = 0.20949721, Time = 0.1396648
Agent 2: Allocation = 1.0, Price = 0.63407821, Time = 0.0
Agent 3: Allocation = 1.0, Price = 0.63407821, Time = 0.0
Agent 4: Allocation = 0.76815642, Price = 0.0, Time = 0.76815642
Agent 5: Allocation = 0.83798883, Price = 0.20949721, Time = 0.1396648
Agent 6: Allocation = 0.0, Price = 0.0, Time = 0.0
Agent 7: Allocation = 0.55865922, Price = 0.069832402, Time = 0.0
Agent 8: Allocation = 0.55865922, Price = 0.069832402, Time = 0.0
Agent 9: Allocation = 0.76815642, Price = 0.0, Time = 0.76815642


In [43]:
# Store optimal values 
allocations = []
prices = []
times = []
for i in range(N):
    allocations.append(x[i].varValue)
    prices.append(p[i].varValue)
    times.append(t[i].varValue)

In [49]:
# Check IR and IC constraints
eps = 1e-4
for i in range(N):
    util_i = agents[i][0] * allocations[i] - agents[i][1] * prices[i] - times[i]
    if util_i + eps < 0: print(f"IR violated at {i}")
    for j in range(N):
        if i != j:
            util_j = agents[i][0] * allocations[j] - agents[i][1] * prices[j] - times[j]
            if util_i + eps < util_j:
                print(f"IC violated at ({i}, {j})")
                print(f"Utility of i: {util_i}; Utility of j: {util_j}")


In [45]:
agents

array([[ 3.,  4.],
       [ 5.,  4.],
       [ 7.,  3.],
       [ 8.,  2.],
       [ 7., 10.],
       [ 6.,  5.],
       [ 1.,  8.],
       [ 1.,  1.],
       [ 2.,  5.],
       [ 6.,  7.]])